In [2]:
import sqlite3
import datetime
import pandas as pd
conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
query = "select distinct Ticker from stock_table_hr"
r_df = pd.read_sql(query,conn)
list_db = r_df['Ticker'].values.tolist()
print(list_db)
print(len(list_db))

['AAV.BK', 'ACE.BK', 'ADVANC.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BAM.BK', 'BANPU.BK', 'BBL.BK', 'BCH.BK', 'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BLA.BK', 'BTS.BK', 'BYD.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DELTA.BK', 'DOHOME.BK', 'DTAC.BK', 'EA.BK', 'EGCO.BK', 'EPG.BK', 'ESSO.BK', 'FORTH.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'GUNKUL.BK', 'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JAS.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 'KEX.BK', 'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MEGA.BK', 'MINT.BK', 'MTC.BK', 'NEX.BK', 'ONEE.BK', 'OR.BK', 'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PSL.BK', 'PTG.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 'QH.BK', 'RATCH.BK', 'RBF.BK', 'RCL.BK', 'SABUY.BK', 'SAWAD.BK', 'SCB.BK', 'SCC.BK', 'SCGP.BK', 'SINGER.BK', 'SPALI.BK', 'SPRC.BK', 'STA.BK', 'STARK.BK', 'STGT.BK', 'TCAP.BK', 'THANI.BK', 'THG.BK', 'TIDLOR.BK', 'TIPH.BK', 'TISCO.B

In [1]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output, callback ,State
import plotly.express as px
import pandas as pd

import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

def plot(ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # ticker = input("Enter tickers")
    query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '2023-01-01' and datetime < '2023-01-06'" % ticker
    query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
    query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
    Hr = pd.read_sql(query1,conn)
    Day = pd.read_sql(query2,conn)
    Mo = pd.read_sql(query3,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
                row_width=[0.2, 0.7])
    # Day['time'] = pd.to_datetime(Day['Datetime'])
    # Day.set_index(Day['time'],inplace = True)

    #Candlestick
    for column in [Hr,Day,Mo]:
        # name = column.Ticker[0]
        column['time'] = pd.to_datetime(column['Datetime'])
        column['MA50'] = column['Close'].rolling(window = 50 , min_periods = 0).mean()
        column['MA200'] = column['Close'].rolling(window = 200 , min_periods = 0).mean()
        column.set_index(column['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=column.Datetime,
                        open=column.Open,
                        high=column.High,
                        low=column.Low,
                        close=column.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=column.Datetime,y=column['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=column.Datetime,y=column['MA200']),row=1, col=1)
        fig.add_trace(go.Bar(x=column.Datetime, y=column.Volume,showlegend=False), row=2, col=1)
        
    df_resample_hr = Hr.resample('H').max()
    merged_index_hr  = Hr.index.append(df_resample_hr.index)
    timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

    df_resample_day = Day.resample('D').max()
    merged_index_day  = Day.index.append(df_resample_day.index)
    timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])

    fig.update_xaxes(rangebreaks=[dict(values=timegap_day),
                                 dict(values=timegap_hr, dvalue=3600000)])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active=0,
            buttons=list(
                [dict(label = 'Disable',
                    method = 'update',
                    args = [{'visible': [False, False, False]}, # the index of True aligns with the indices of plot traces
                            {'title': 'Disable',
                            'showlegend':True}]),
                dict(label = 'Hour',
                    method = 'update',
                    args = [{'visible': [True, True, True, True, False, False, False, False, False, False, False, False]},
                            {'title': '%s Hour' % ticker,
                            'showlegend':True}]),
                dict(label = 'Day',
                    method = 'update',
                    args = [{'visible': [False, False, False, False, True, True, True, True, False, False, False, False]},
                            {'title': '%s Day' % ticker,
                            'showlegend':True}]),
                dict(label = 'Month',
                    method = 'update',
                    args = [{'visible': [False, False, False, False, False, False, False, False, True, True, True, True]},
                            {'title': '%s Mount' % ticker,
                            'showlegend':True}]),
                ])
            )
        ])
    # fig.update_yaxes(rangemode="nonnegative")
    # fig.update_xaxes(rangemode="nonnegative")
    fig.update_traces(visible="legendonly")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def stock_name():
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query = "select distinct Ticker from stock_table_hr"
    r_df = pd.read_sql(query,conn)
    list_db = r_df['Ticker'].values.tolist()
    return list_db


#DASH
app.layout = html.Div(children=[
    html.H1(children='Stock Radar'),
    html.Div([
        "Stock : ",
        dcc.Input(id='ticker-input', value='', type='text'),
        html.Button(id='submit-button', n_clicks=0, children='Submit'),
    ]),
    html.Div(id='output-div'),

    dcc.Graph(
        id='graph',
        figure=plot('')
    ),
    dcc.ConfirmDialog(
        id='popup',
        message='Not found that tickers',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_dl',
        message='Downloaded',
        displayed=False,
    ),
])

@app.callback([Output('graph','style'),Output('popup', 'displayed')],
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])

def update_popup_and_graph(n_clicks, value):
    list_db = stock_name()
    value = value.upper()+ ".BK"
    if n_clicks:
        if value.strip() == '' or (value.strip() not in list_db):
            return plot(''),True
        else:
            return plot(value),False
    return plot(''),False


@app.callback(Output('graph','figure'),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])
def update_output(n_clicks, value):
    list_db = stock_name()
    value = value.upper()+ ".BK"
    if value.strip() == '' or (value.strip() not in list_db):
            fig = plot('')
            return fig
    else:
        value = value.upper()
        fig = plot(value)
        return fig

@app.callback(
    Output("popup_dl", "displayed"),
    [Input("popup", "submit_n_clicks")],
    [State('ticker-input', 'value')]
)
def execute_function(submit_n_clicks ,value):
    if submit_n_clicks:
        return True
    else:
        pass
    return False


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


PyQT

In [2]:
import sys

from PyQt5.QtWebEngineWidgets import QWebEnginePage
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEngineView

url = 'http://127.0.0.1:8050/'

app = QApplication(sys.argv)

# QWebEngineView
browser = QWebEngineView()
browser.load(QUrl(url))
browser.show()

sys.exit(app.exec_())

SystemExit: 0

c:\Users\1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [1]:
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
import plotly.express as px


class Widget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.button = QtWidgets.QPushButton('Plot', self)
        self.browser = QtWebEngineWidgets.QWebEngineView(self)

        vlayout = QtWidgets.QVBoxLayout(self)
        vlayout.addWidget(self.button, alignment=QtCore.Qt.AlignHCenter)
        vlayout.addWidget(self.browser)

        self.button.clicked.connect(self.show_graph)
        self.resize(1000,800)

    def show_graph(self):
        df = px.data.tips()
        fig = px.box(df, x="day", y="total_bill", color="smoker")
        fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
        self.browser.setHtml(fig.to_html(include_plotlyjs='cdn'))

if __name__ == "__main__":
    app = QtWidgets.QApplication([])
    widget = Widget()
    widget.show()
    app.exec()
    